In [1]:
#!pip install datasets==3.6.0
#!pip install peft
!pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; plat

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

from datasets import Dataset
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import TrainingArguments, Trainer

from peft import get_peft_model, LoraConfig, TaskType

In [3]:
reviews = pd.read_csv("imdb.csv").sample(500)
reviews.head()

,review,sentiment
4047,The Contaminated Man is a good film that has a...,positive
3745,At school I was taught how some shots were cal...,positive
1721,"I'm giving it only 9 out of 10, because I need...",positive
1267,I liked this movie I remember there was one ve...,positive
2661,This movie is excellent ( i watched the French...,positive


In [4]:
reviews = reviews.rename(columns={"review": "text", "sentiment": "label"})
reviews['label'] = LabelEncoder().fit_transform(reviews['label'])
reviews.head()

,text,label
4047,The Contaminated Man is a good film that has a...,1
3745,At school I was taught how some shots were cal...,1
1721,"I'm giving it only 9 out of 10, because I need...",1
1267,I liked this movie I remember there was one ve...,1
2661,This movie is excellent ( i watched the French...,1


To help with this task, we're going to use the [Datasets library](https://huggingface.co/docs/datasets/en/index), which allows for working with various types of datasets, from text, to audio, to images.

First, convert the reviews DataFrame into a dataset object by using the [from_pandas function](https://huggingface.co/docs/datasets/en/index).

In [5]:
# dataset = Dataset.from_pandas(df)
reviews_dataset = Dataset.from_pandas(reviews)

Once converted, we can perform a train/test split using a method of the Dataset object.

Peform an 80/20 train/test split using the [train_test_split method](https://huggingface.co/docs/datasets/en/index). Save the result back to the same Dataset object.

In [6]:
reviews_dataset = reviews_dataset.train_test_split(test_size=0.2, shuffle=True)

In [7]:
reviews_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 400
    })
    test: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 100
    })
})

Now, extract the training and test portion into separate Datasets. Name these new datasets train_dataset and test_dataset, respectively.

In [8]:
train_dataset = reviews_dataset.get("train")
test_dataset = reviews_dataset.get("test")

We're going to be working with a DistilBERT model, which means that we'll need to tokenize our input in the way that DistilBERT expects. For this, we can use the [DistilBertTokenizerFast](https://huggingface.co/docs/transformers/en/model_doc/distilbert?usage=AutoModel#transformers.DistilBertTokenizerFast) tokenizer.

In [9]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Then we'll apply our tokenizer to the training and test datsets using the map method.

In [10]:
train_dataset = train_dataset.map(lambda df: tokenizer(df['text'], padding="max_length", truncation=True), batched=True)
test_dataset = test_dataset.map(lambda df: tokenizer(df['text'], padding="max_length", truncation=True), batched=True)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Finally, we'll set the format to torch so that we're working with [PyTorch](https://pytorch.org/) tensors and only extract the columns that we need.

In [11]:
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Now, let's load in the pretrained DistilBERT model.

### Part 1: Fine-tuning All Parameters

In [12]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Now, we need to set up a [Trainer](https://huggingface.co/docs/transformers/en/main_classes/trainer) for our model.

First, create a [TraininingArguments](https://huggingface.co/docs/transformers/v4.52.3/en/main_classes/trainer#transformers.TrainingArguments) object. Set num_train_epochs to 5, weight_decay to 0.01, and report_to = 'none'

In [13]:
training_args = TrainingArguments(
    num_train_epochs=5,
    weight_decay=0.01,
    report_to = 'none'
)

Finally, create a Trainer object using the model, the Training Arguments that you created, and with the train_dataset equal to train_dataset.

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

Now, use the [train method](https://huggingface.co/docs/transformers/en/main_classes/trainer#transformers.Trainer.train) on your Trainer object.

In [15]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=250, training_loss=0.22334117126464845, metrics={'train_runtime': 100.3558, 'train_samples_per_second': 19.929, 'train_steps_per_second': 2.491, 'total_flos': 264934797312000.0, 'train_loss': 0.22334117126464845, 'epoch': 5.0})

Once the model has been fit, use the [predict method](https://huggingface.co/docs/transformers/en/main_classes/trainer#transformers.Trainer.predict) of the Trainer object to generate a set of predictions on the test_dataset.

In [16]:
predictions = trainer.predict(test_dataset)

Extract the actual test labels and the predicted labels from the predictions. Note that both the true labels and predicted probabilites are contained as an attribute of the predictions.

In [17]:
predicted_labels = np.argmax(predictions.predictions, axis=-1)
true_labels = predictions.label_ids

Finally, look the confusion matrix and classification report for these predictions.

In [18]:
print(classification_report(true_labels, predicted_labels))

              precision    recall  f1-score   support

           0       0.87      0.87      0.87        47
           1       0.89      0.89      0.89        53

    accuracy                           0.88       100
   macro avg       0.88      0.88      0.88       100
weighted avg       0.88      0.88      0.88       100



In [19]:
confusion_matrix(true_labels, predicted_labels)

array([[41,  6],
       [ 6, 47]])

### Part 2: Training Only a Subset of the Parameters

Let's first reload the pretrained distilbert model.

Then, we'll make none of the parameters trainable by setting the `requires_grad` attribute to False.

In [20]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

for param in model.distilbert.parameters():
    param.requires_grad = False

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Then, we'll go back and make the last 2 layers trainable.

In [21]:
for i in [4, 5]:
    for param in model.distilbert.transformer.layer[i].parameters():
        param.requires_grad = True

We'll now set up TrainingArguments and a Trainer as before to train the model.

In [22]:
training_args = TrainingArguments(
    num_train_epochs=5,
    weight_decay=0.01,
    report_to = 'none'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

trainer.train()

Step,Training Loss


TrainOutput(global_step=250, training_loss=0.27631170654296877, metrics={'train_runtime': 50.7431, 'train_samples_per_second': 39.414, 'train_steps_per_second': 4.927, 'total_flos': 264934797312000.0, 'train_loss': 0.27631170654296877, 'epoch': 5.0})

How do the results of fine-tuning only the last two layers compare to fine-tuning all layers? How does the training time compare?

In [23]:
predictions = trainer.predict(test_dataset)

predicted_labels = np.argmax(predictions.predictions, axis=-1)
true_labels = predictions.label_ids

print(classification_report(true_labels, predicted_labels))
print(confusion_matrix(true_labels, predicted_labels))

              precision    recall  f1-score   support

           0       0.88      0.81      0.84        47
           1       0.84      0.91      0.87        53

    accuracy                           0.86       100
   macro avg       0.86      0.86      0.86       100
weighted avg       0.86      0.86      0.86       100

[[38  9]
 [ 5 48]]


### Part 3: Parameter-Efficient Fine-Tuning

Now, let's see how we can use the [peft library](https://huggingface.co/docs/peft/en/index) to more efficiently fine-tune our model.

First, we'll re-initalize the model.

In [24]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Create a [LoraConfig object](https://huggingface.co/docs/peft/en/index) were you set the Lora attention dimension to 8, the lora_alpha to 16, the lora_dropout to 0.1, the target_modules to "q_lin" and "v_lin" (these are the "query" and "value" projections), and set the task_type to TaskType.SEQ_CLS.

In [25]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_lin", "v_lin"],
    task_type=TaskType.SEQ_CLS
)

Now, use the [get_peft_model function](https://huggingface.co/docs/peft/v0.15.0/en/package_reference/peft_model#peft.get_peft_model) to create the Lora model pass in the distilbert model and the LoraConfig object that you created.

In [26]:
model = get_peft_model(model = model, peft_config = lora_config)

How many trainable parameters does the resulting model have? Hint: you can use the [print_trainable_parameters function](https://huggingface.co/docs/peft/v0.15.0/en/package_reference/peft_model#peft.PeftModel.print_trainable_parameters).

In [27]:
model.print_trainable_parameters()

trainable params: 739,586 || all params: 67,694,596 || trainable%: 1.0925


We'll again set up the same TrainingArguments and create our Trainer object to grain the model.

In [28]:
training_args = TrainingArguments(
    num_train_epochs=5,
    weight_decay=0.01,
    report_to = 'none'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

trainer.train()

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss


TrainOutput(global_step=250, training_loss=0.63392333984375, metrics={'train_runtime': 66.2627, 'train_samples_per_second': 30.183, 'train_steps_per_second': 3.773, 'total_flos': 269478813696000.0, 'train_loss': 0.63392333984375, 'epoch': 5.0})

How does the performance of the lora model compare to the previous two? How does the training time compare?

In [29]:
predictions = trainer.predict(test_dataset)

predicted_labels = np.argmax(predictions.predictions, axis=-1)
true_labels = predictions.label_ids

print(classification_report(true_labels, predicted_labels))
print(confusion_matrix(true_labels, predicted_labels))

              precision    recall  f1-score   support

           0       0.89      0.66      0.76        47
           1       0.75      0.92      0.83        53

    accuracy                           0.80       100
   macro avg       0.82      0.79      0.79       100
weighted avg       0.82      0.80      0.80       100

[[31 16]
 [ 4 49]]
